In [3]:
import os
import xlsxwriter
import numpy as np
import pandas as pd
from pandas import isnull

In [4]:
def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n += 1 
            return n
        else:
            continue


def InitialRun(S):
    """This takes S a list of filenames and returns the first run index"""
    last_int_index = FindDel(S) -1
    first_int_index = 0 
    for i in S:                                                          
        if i[0:3] == 'ENG':
            for j in range(last_int_index):
                try:
                    int(i[j])
                    run_number = i[first_int_index:last_int_index]
                    break
                except:
                    first_int_index += 1
        else:
            continue
        break
    return((run_number))

def CycleRuns(S):
    """This takes a list of filenames and returns a list of all the runs in the file"""
    Start_run = int(InitialRun(S))
    Last_run = int(InitialRun(reversed(S)))
    Runs = range(Start_run,Last_run)
    return (list(Runs))

def RunIndex(s,n): #ENGINX00305258
    """s is a string in the for m of a filename from the enginx data and returns it's run number, n is the FindDel"""
    last_int_index = n-1
    first_int_index = 0
    for j in range(last_int_index):
        try:
            int(s[j])
            return s[first_int_index:last_int_index]
        except:
            first_int_index += 1

In [5]:
path = "C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx"
def Translator(path):
    xls = pd.ExcelFile(path)
    df = xls.parse(sheet_name = "Sheet1")
    df = df[["Filename", "Column_name"]]
    df = df.fillna(0)
    true_name_translater = dict(zip(df["Filename"],df["Column_name"]))
    s = set(true_name_translater.values())
    col_names = list(s)
    col_names.remove(0)
    col_names.append("measured_temperature_U")
    col_names.append("unregistered_value")
    return true_name_translater, col_names

true_name_translater, col_names = Translator(path)

In [6]:
def AddRawData(run_number):
    file_path = output+"\\"+foldername+"\\"+str(run_number)+".txt"
    output_file = open(file_path, "r")
    contents = output_file.read()
    split_contents = contents.split("kamehameha") # Had to pick somthing that wouldn't be in data
    del split_contents[-1]
    clean_split_contents = []
    for i,j in zip(split_contents, data):
        a = i.replace("\n","").replace("[","").replace("]","")
        a = a.split(" ")
        a = list(filter(None, a))
        if "data" in j:
            clean_split_contents.append(j[0:16]+str(run_number))
            Neutron_file = open(Neutron_folder+"\\"+j[0:16]+str(run_number)+".txt", "w+")
            Neutron_file.write(str(a))
            Neutron_file.close()
        elif "time" in j:
            untdate = i.replace('T',' ')
            clean_split_contents.append(untdate)
        else:
            clean_split_contents.append(a)
    raw_data = dict(zip(data, list(clean_split_contents)))
    output_file.close()
    os.remove(file_path) 
    return raw_data

In [33]:
def NewRun(run_number, run_data, time_list, raw_data, file_location):
    """This activates if a new run has started, it makes a new list of dictionaries and makes a CSV out of current data"""
    if type(time_list) == int:
        df1 = pd.DataFrame()
        df1["Date/Time"] = raw_data["neutron_start_time"]
        for i in raw_data:
            df1[str(i)] = str(raw_data[i])
        print(run_number + " in " + file_location)
        df1.to_csv(file_location+"\\"+run_number)
    else:
        df1 = pd.DataFrame()
        for i in time_list:
            row = np.array([i])
            column_names = np.array(["Date/Time"])
            for j in run_data:
                try:
                    row = np.append(row, [j[1][i]])
                    column_names = np.append(column_names, [j[0]])
                except:
                    pass
            try:
                df2 = pd.DataFrame([row], columns = column_names)
                df1 = df1.append(df2, sort=False)
            except:
                df2 = pd.DataFrame([row][:-1], columns = column_names)
                df1 = df1.append(df2, sort=False)
        for i in raw_data:
            df1[str(i)] = str(raw_data[i])
        print(run_number + " in " + file_location)
        df1.to_csv(file_location+"\\"+run_number)
    
    raw_data = {}
    run_data = []
    time_list = np.array([])
    for i,j in enumerate(col_names):
        run_data.append((j,{}))
    
    return run_data,time_list,raw_data

def Find_dict(run_data, trans_filename):
    for i,j in enumerate(run_data):
        if j[0] == trans_filename:
            return i

def AddData(file_path, filename, n, run_data, time_list):
    """This takes data from a txt file and puts it into a list of times and the dictionaries"""
    df = pd.read_csv(file_path, sep = "\t", names = ["Date/Time", str(true_name_translater[filename[n:]])])
    file_times = df["Date/Time"].values
    file_data = df[str(true_name_translater[filename[n:]])].values 
    dict_index = Find_dict(run_data, true_name_translater[filename[n:]])
    if os.stat(file_path).st_size == 0: # remove empty files
        pass
    elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
        pass
    elif file_times[0][10] =="T":
        for i in range(len(file_times)): # fix the T issue in some times 
            untdate = file_times[0].replace('T',' ')
            file_times = np.append(file_times,[untdate])
            file_times = np.delete(file_times,0, axis = None)
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:#Alot of tempreture files, may be some overlap in time
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:#rest of the files 
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    else: # Normal time columns
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    return run_data, time_list

In [35]:
#%%timeit
#%prun
directory = r"C:\Users\zahaab\test_case4"
output = directory + "\\temp_raw_data"

Neutron_folder = CreateFolder(directory+"\\"+"Neutron_data")
ENGINX_DATA = CreateFolder(directory+"\\"+"clean_data")
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"]

for foldername in os.listdir(directory):
    print(foldername)
    if foldername[0:5] == 'cycle':
        folder_path = directory+"\\"+foldername
        clean_data = CreateFolder(ENGINX_DATA+"\\"+foldername)
        n = FindDel(os.listdir(folder_path))
        filenames = os.listdir(folder_path)
        raw_data = {}
        run_data = []
        for i,j in enumerate(col_names):
            run_data.append((j,{}))
        time_list = np.array([])
        finale_file = 0
        run_index = RunIndex(filenames[0],n)
        
        for filename in reversed(filenames):  
            if filename[0:3] == 'ENG':
                finale_file = filename
                break
        
        for k, filename in enumerate(filenames):
            print(filename)
            if k == 0:
                continue
                
            elif "Copy" in filename:
                continue
                
            elif "copy" in filename:
                continue
                
            if filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    continue
                except ValueError: 
                    pass
            
            print(RunIndex(filenames[k],n))
            print(filename[-3:])
            
            file_path = folder_path+'\\'+filename
            if true_name_translater[filename[n:]] == 0:
                pass
                
            elif filename[-3:] == "nxs":
                pass
            
            elif RunIndex(filenames[k],n) != run_index:
                run_index = RunIndex(filenames[k],n)
                if len(time_list) == 0:
                    pass
                else:
                    run_data,time_list,raw_data = NewRun(RunIndex(filenames[k-1],n), run_data, time_list, raw_data, clean_data)
                    if filename[-3:].lower() in ["RAW", "raw", "Raw"]:
                        raw_data = AddRawData(RunIndex(filenames[k],n))
                    elif filename[-3:] == "txt":
                        run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                    else:
                        pass
            else:
                if filename[-3:].lower() in ["RAW", "raw", "Raw"]:
                    raw_data = AddRawData(RunIndex(filenames[k],n))
                elif filename[-3:] == "txt":
                    run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                else:
                    pass
            
            if filename == finale_file:
                break
        NewRun(RunIndex(finale_file,n), run_data, time_list, raw_data, clean_data)

clean_data
cycle_04_1
ENG44008.RAW
ENG44008_Theta.txt
44008
txt
ENG44008_X.txt
44008
txt
ENG44008_Y.txt
44008
txt
ENG44008_Z.txt
44008
txt
ENG44009.RAW
44009
RAW
44008 in C:\Users\zahaab\test_case4\clean_data\cycle_04_1
ENG44009_Theta.txt
44009
txt
ENG44009_X.txt
44009
txt
ENG44009_Y.txt
44009
txt
ENG44009_Z.txt
44009
txt
ENG44010.RAW
44010
RAW
44009 in C:\Users\zahaab\test_case4\clean_data\cycle_04_1
ENG44010_Theta.txt
44010
txt
ENG44010_X.txt
44010
txt
ENG44010_Y.txt
44010
txt
ENG44010_Z.txt
44010
txt
ENG44011.RAW
44011
RAW
44010 in C:\Users\zahaab\test_case4\clean_data\cycle_04_1
ENG44011_Theta.txt
44011
txt
ENG44011_X.txt
44011
txt
ENG44011_Y.txt
44011
txt
ENG44011_Z.txt
44011
txt
ENG44012.RAW
44012
RAW
44011 in C:\Users\zahaab\test_case4\clean_data\cycle_04_1
ENG44012_Theta.txt
44012
txt
ENG44012_X.txt
44012
txt
ENG44012_Y.txt
44012
txt
ENG44012_Z.txt
44012
txt
ENG44013.RAW
44013
RAW
44012 in C:\Users\zahaab\test_case4\clean_data\cycle_04_1
ENG44013_Theta.txt
44013
txt
ENG44013_X.

ENGINX00204005_cyba_a.txt
00204005
txt
ENGINX00204005_cyba_b.txt
00204005
txt
ENGINX00204005_cyba_c.txt
00204005
txt
ENGINX00204005_ICPdebug.txt
00204005
txt
ENGINX00204005_ICPevent.txt
00204005
txt
ENGINX00204005_slitpos.txt
00204005
txt
ENGINX00204005_Status.txt
00204005
txt
ENGINX00204005_w.txt
00204005
txt
ENGINX00204005_x.txt
00204005
txt
ENGINX00204005_y.txt
00204005
txt
ENGINX00204005_z.txt
00204005
txt
ENGINX00204006.nxs
00204006
nxs
ENGINX00204006.raw
00204006
raw
00204006 in C:\Users\zahaab\test_case4\clean_data\cycle_13_1
ENGINX00204006_cyba_a.txt
00204006
txt
ENGINX00204006_cyba_b.txt
00204006
txt
ENGINX00204006_cyba_c.txt
00204006
txt
ENGINX00204006_ICPdebug.txt
00204006
txt
ENGINX00204006_ICPevent.txt
00204006
txt
ENGINX00204006_Slit_H.txt
00204006
txt
ENGINX00204006_Status.txt
00204006
txt
ENGINX00204006_w.txt
00204006
txt
ENGINX00204006_x.txt
00204006
txt
ENGINX00204006_y.txt
00204006
txt
ENGINX00204006_z.txt
00204006
txt
ENGINX00204007.nxs
00204007
nxs
ENGINX00204007.r

00269856 in C:\Users\zahaab\test_case4\clean_data\cycle_17_1
ENGINX00269856_crosshead_pos.txt
00269856
txt
ENGINX00269856_ICPdebug.txt
00269856
txt
ENGINX00269856_ICPevent.txt
00269856
txt
ENGINX00269856_Status.txt
00269856
txt
ENGINX00269856_strain.txt
00269856
txt
ENGINX00269856_stress.txt
00269856
txt
ENGINX00269856_w.txt
00269856
txt
ENGINX00269856_x.txt
00269856
txt
ENGINX00269856_y.txt
00269856
txt
ENGINX00269856_z.txt
00269856
txt
ENGINX00269857.log
00269857
log
ENGINX00269857.nxs
00269857
nxs
ENGINX00269857.raw
00269857
raw
00269857 in C:\Users\zahaab\test_case4\clean_data\cycle_17_1
ENGINX00269857_crosshead_pos.txt
00269857
txt
ENGINX00269857_furnace.txt
00269857
txt
ENGINX00269857_furnace_pwr.txt
00269857
txt
ENGINX00269857_ICPdebug.txt
00269857
txt
ENGINX00269857_ICPevent.txt
00269857
txt
ENGINX00269857_Lamp.txt
00269857
txt
ENGINX00269857_slitpos.txt
00269857
txt
ENGINX00269857_slit_h.txt
00269857
txt
ENGINX00269857_slit_v.txt
00269857
txt
ENGINX00269857_Status.txt
00269857

00269868 in C:\Users\zahaab\test_case4\clean_data\cycle_17_1
ENGINX00269868_crosshead_pos.txt
00269868
txt
ENGINX00269868_furnace.txt
00269868
txt
ENGINX00269868_furnace_pwr.txt
00269868
txt
ENGINX00269868_ICPdebug.txt
00269868
txt
ENGINX00269868_ICPevent.txt
00269868
txt
ENGINX00269868_Lamp.txt
00269868
txt
ENGINX00269868_slitpos.txt
00269868
txt
ENGINX00269868_slit_h.txt
00269868
txt
ENGINX00269868_slit_v.txt
00269868
txt
ENGINX00269868_Status.txt
00269868
txt
ENGINX00269868_strain.txt
00269868
txt
ENGINX00269868_stress.txt
00269868
txt
ENGINX00269868_w.txt
00269868
txt
ENGINX00269868_x.txt
00269868
txt
ENGINX00269868_y.txt
00269868
txt
ENGINX00269868_z.txt
00269868
txt
ENGINX00269869.log
00269869
log
ENGINX00269869.nxs
00269869
nxs
ENGINX00269869.raw
00269869
raw
00269869 in C:\Users\zahaab\test_case4\clean_data\cycle_17_1
ENGINX00269869_crosshead_pos.txt
00269869
txt
ENGINX00269869_furnace.txt
00269869
txt
ENGINX00269869_furnace_pwr.txt
00269869
txt
ENGINX00269869_ICPdebug.txt
0026

ENGINX00305262_amplitude.txt
00305262
txt
ENGINX00305262_C6_master_freq.txt
00305262
txt
ENGINX00305262_C6_master_phase.txt
00305262
txt
ENGINX00305262_C6_slave_freq.txt
00305262
txt
ENGINX00305262_C6_slave_phase.txt
00305262
txt
ENGINX00305262_C9_master_freq.txt
00305262
txt
ENGINX00305262_C9_master_phase.txt
00305262
txt
ENGINX00305262_C9_slave_freq.txt
00305262
txt
ENGINX00305262_C9_slave_phase.txt
00305262
txt
ENGINX00305262_control_channel.txt
00305262
txt
ENGINX00305262_control_channel_string.txt
00305262
txt
ENGINX00305262_Cyba_a.txt
00305262
txt
ENGINX00305262_Cyba_b.txt
00305262
txt
ENGINX00305262_Cyba_c.txt
00305262
txt
ENGINX00305262_cycles.txt
00305262
txt
ENGINX00305262_cycle_rbv.txt
00305262
txt
ENGINX00305262_frequency.txt
00305262
txt
ENGINX00305262_ICPdebug.txt
00305262
txt
ENGINX00305262_ICPevent.txt
00305262
txt
ENGINX00305262_ICPputlog.txt
00305262
txt
ENGINX00305262_jaws1_hgap.txt
00305262
txt
ENGINX00305262_jaws2_hgap.txt
00305262
txt
ENGINX00305262_jaws3_hgap.txt

ENGINX00305265_C9_slave_freq.txt
00305265
txt
ENGINX00305265_C9_slave_phase.txt
00305265
txt
ENGINX00305265_control_channel.txt
00305265
txt
ENGINX00305265_control_channel_string.txt
00305265
txt
ENGINX00305265_Cyba_a.txt
00305265
txt
ENGINX00305265_Cyba_b.txt
00305265
txt
ENGINX00305265_Cyba_c.txt
00305265
txt
ENGINX00305265_cycles.txt
00305265
txt
ENGINX00305265_cycle_rbv.txt
00305265
txt
ENGINX00305265_frequency.txt
00305265
txt
ENGINX00305265_ICPdebug.txt
00305265
txt
ENGINX00305265_ICPevent.txt
00305265
txt
ENGINX00305265_ICPputlog.txt
00305265
txt
ENGINX00305265_jaws1_hgap.txt
00305265
txt
ENGINX00305265_jaws2_hgap.txt
00305265
txt
ENGINX00305265_jaws3_hgap.txt
00305265
txt
ENGINX00305265_jaws3_vgap.txt
00305265
txt
ENGINX00305265_Max_output_1.txt
00305265
txt
ENGINX00305265_Max_output_2.txt
00305265
txt
ENGINX00305265_Max_output_3.txt
00305265
txt
ENGINX00305265_North_X.txt
00305265
txt
ENGINX00305265_North_X_hlim.txt
00305265
txt
ENGINX00305265_North_X_llim.txt
00305265
txt
ENG

Neutron_data
OtherData
temp_raw_data


In [84]:
def CombineRigCleanData(cycle):
    direct = "E:\\stuff\\OtherData\\"+cycle+"\\Stress Rig"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 

    df3 = pd.DataFrame()
    for filename in os.listdir(direct):
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line = output_file.readline()
        constants = {"Cross_Sectional_Area" : first_line[:33].split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : first_line[33:].split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")

    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})

    #df3.set_index("Date/Time")
    #df3 = df3.loc[~df3.index.duplicated(keep='first')] #drop duplicates 
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')

    return(result) 

def CleanInstron01(cycle, filenames, clean_file_time):
    direct = r"E:\stuff\OtherData\INSTRON_01"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 
    
    #find the files that needed for the spesific cycle
    start_file_index, end_file_index = 0, 0
    for i in clean_file_time:
        if i in time_list:
            start_file_index = clean_file_time.index(i)-1
            break 
    for i in clean_file_time.reverse():
        if i in time_list:
            end_file_index = (clean_file_time.index(i)-1)*-1
            break 
    
    df3 = pd.DataFrame()
    for filename in os.listdir(direct)[start_file_index, end_file_index+1]:
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "\t", skipinitialspace = True, skiprows = [0,1,2,3])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line, second_line, third_line = output_file.readline(), output_file.readline(1), output_file.readline(2)
        constants = {"Cross_Sectional_Area" : first_line.split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : second_line.split("=")[-1].replace(" ", ""),
                     "RB Number" : third_line.split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")
        
    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    
    unttime = []
    for i in df3["Date/Time"].values:
        untdate = "{}-{}-{} {}:{}:{}".format(
            date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        unttime.append(untdate)
    df3 = df3.drop(columns=["Date/Time"])
    df3["Date/Time"] = unttime
    
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')
    return(result) 

a


In [ ]:
ENGINX_DATA = r"E:\clean_data"

instron_start = len(os.listdir(ENGINX_DATA))

filenames = os.listdir(r"E:\stuff\OtherData\INSTRON_01")
clean_file_time = []
for filename in filenames:
    if "continuous" in filename:
        filenames.remove(filename)
    else: 
        date = filename[11:]
        untdate = "{}-{}-{} {}:{}:{}".format(
        date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        clean_file_time.append(untdate)
            
for i,j in  enumerate(os.listdir(ENGINX_DATA)):
    if j == "cycle_17_3":
        CombineRigCleanData(j)
        instron_start = i
    elif i > instron_start:
        CleanInstron01(j, filenames, clean_file_time)
    else:
        CombineRigCleanData(j)

[1, 2, 3]
